# NMF - No watch times
No watch times - only adjacency **user - (tag) - author** matrix.

## Init

In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from functools import partial

import plotly.express as px


# import implicit # Matrix Factorization

from sklearn.decomposition import NMF
from sklearn.model_selection import KFold
from cmfrec import CMF_implicit

In [2]:
import sys
sys.path.append("../")
from src.data_preprocessing import TrainTestGenerator
from src.evaluator import Evaluator

In [3]:
show_ploty = False

In [4]:
data_dir = "../data/"
data_generator = TrainTestGenerator(data_dir)

## Model

In [5]:
# Model wrapper

class CMF_recommender:
    def __init__(self, k=50):
        self.model = CMF_implicit(
            # verbose=False,
            # method="als",
            k=k,
            nonneg=True,
            random_state=1,
            # lambda_=1e+1
        )

    def fit(self, data: pd.DataFrame):
        data = data.copy()
        data["weight"] = 1  # Binary adjacency matrix (no weights)
        data = data.rename(columns={
            "userID": "UserId",
            "artistID": "ItemId",
            "weight": "Rating"
        })
        self.model.fit(data)

    def recommend(self, user_id, n):
        recommendations = self.model.topN(user_id, n=n)
        return recommendations

## Evaluation

In [6]:
# Evaluator (forward chaining)

evaluator = Evaluator(CMF_recommender, data_generator)
evaluator.evaluate()

evaluator.save_results("../results/cmf_binary_ranks.csv", "../results/cmf_binary_times.csv")

In [7]:
# Hit Rate

evaluator.get_hit_rates()

,cases,5,10,25,50,500
2008,4556,0.017120,0.025461,0.039069,0.066945,0.295654
2009,4687,0.016215,0.025389,0.043311,0.062940,0.240239
2010,6133,0.019240,0.030980,0.054623,0.084950,0.262188
2011,1129,0.047830,0.078831,0.132861,0.186891,0.434898


In [8]:
# Mean Reciprocal Rank

evaluator.get_mrr()

,cases,mrr
2008,2608,0.023833
2009,3086,0.020357
2010,4306,0.021187
2011,878,0.046704


In [9]:
# Times

evaluator.get_times()

,count,mean,std,min,25%,50%,75%,max
task,,,,,,,,
model_fit,4.0,0.344390,0.131519,0.262868,0.273549,0.287073,0.357914,0.540546
model_init,4.0,0.000022,0.000009,0.000014,0.000018,0.000019,0.000023,0.000035
recommend_user,2622.0,0.000730,0.000359,0.000343,0.000553,0.000696,0.000851,0.013427


In [10]:
evaluator.get_fit_per_year_times()

,tag,time
task,,
model_fit,model_fit_2008,0.277109
model_fit,model_fit_2009,0.262868
model_fit,model_fit_2010,0.297037
model_fit,model_fit_2011,0.540546
